In [1]:
import collections
import warnings
from pandas.core.common import SettingWithCopyWarning
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.font_manager
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import forestci as fci
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

Failed to import duecredit due to No module named 'duecredit'


In [10]:
signal = pd.read_csv("resonances0.txt", delim_whitespace=True)
val_data = pd.read_csv("comp.txt", delim_whitespace=True)
signal = signal.replace('-',np.nan)
gamma_test = signal[signal['Gg'].isnull()]
gamma_x_test = gamma_test[['Z','A']]
signal.dropna(inplace=True)

gamma_x_train = signal[['Z','A']]
gamma_y_train = signal[['Gg']]
gamma_x_train = gamma_x_train.values.reshape(-1,2)
gamma_y_train = gamma_y_train.values.ravel()

rfmissgg = RandomForestRegressor(n_estimators=500,max_depth=100,random_state=10,max_features=2)
rfmissgg.fit(gamma_x_train,gamma_y_train)
ymiss_pred = rfmissgg.predict(gamma_x_test)

rfmiss_err = abs(fci.random_forest_error(rfmissgg,gamma_x_train,gamma_x_test,calibrate=False))

gamma_x_test['Gg'] = ymiss_pred
gamma_x_test['dG'] = np.sqrt(rfmiss_err)
gamma_train = signal[['Z','A','Gg','dG']]
gamma_full = pd.concat([gamma_train, gamma_x_test])
gamma_full = gamma_full.sort_values(by=['Z'])

In [3]:
x = gamma_full[['Z','A']]
y = gamma_full[['Gg','dG']]
arr_x = x.values.reshape(-1,2)
arr_y = y.values.reshape(-1,2)
arr_y_pred, arr_y_err = np.hsplit(arr_y,2)
arr_y_err = arr_y_err.ravel()
arr_y_pred = arr_y_pred.ravel()


X_train, X_test, y_train, y_test = train_test_split(arr_x, arr_y, test_size=0.2, random_state=11)

X_train_inc = pd.DataFrame(X_train, columns=['Z','A'])
X_train_inc['A'] = X_train_inc['A'] + 1
arr_X_train_inc = X_train_inc.values.reshape(-1,2)

X_test_inc = pd.DataFrame(X_test, columns=['Z','A'])
X_test_inc['A'] = X_test_inc['A'] + 1
arr_X_test_inc = X_test_inc.values.reshape(-1,2)

y_train, y_train_dd = np.hsplit(y_train,2)
y_test, y_test_dd = np.hsplit(y_test,2)

y_train = y_train.ravel()
y_test = y_test.ravel()
y_test_dd = y_test_dd.ravel()

rf = RandomForestRegressor(n_estimators=500,max_depth=100,random_state=10,max_features=2)
rf.fit(X_train,y_train)
score = rf.score(X_train,y_train)

y_pred = rf.predict(X_test)
rf_err = abs(fci.random_forest_error(rf,X_train,X_test,calibrate=False))
mse = mean_squared_error(y_test, y_pred)


# total_errs = [np.sqrt(rf_err[i] + float(y_test_dd[i])**2) for i in np.arange(len(X_test))]

ratio = [round(float(y_pred[i]) /(float(y_test[i])),4) for i in np.arange(len(y_pred))]
ratio_err = [round(ratio[i]*np.sqrt(float(rf_err[i]) / float(y_pred[i])**2 + (float(y_test_dd[i]) / float(y_test[i]))**2),4) for i in np.arange(len(y_pred))]

In [4]:
with open("data_gamma.txt","w") as f:
    f.write('# Basic Learning Model - Showing Test Case of Average Radiative Width - Score:{0} - MSE: {1} \n'.format(round(score,4),round(mse,4)))
    f.write('#Z   A   Gg   dG   mlGg   mldG   mlGg/gg   d(mlGg/gg)\n')
    for i in range(0, len(y_pred)):
        f.write("{0}   {1}   {2}   {3}   {4}   {5}   {6}   {7}\n".format(arr_X_test_inc[i,0],arr_X_test_inc[i,1],y_test[i],y_test_dd[i],round(y_pred[i],4),round(np.sqrt(rf_err[i]),4),ratio[i],ratio_err[i]))

In [5]:
test_data = pd.read_csv("test.txt", delim_whitespace=True)
test_data = test_data[['Z','A']]
arr_test_data = test_data.values.reshape(-1,2)

test_data_inc = test_data[['A']]+1
arr_test_data_inc = test_data_inc.values.reshape(-1,1)
arr_test_data_inc = arr_test_data_inc.ravel()

rf.fit(arr_x,arr_y_pred)
test_score = rf.score(arr_x,arr_y_pred)
y_pred_test = rf.predict(test_data)
rf_err_test = abs(fci.random_forest_error(rf,arr_x,test_data,calibrate=False))

In [6]:
gg = pd.read_csv("Gg_talys_data.txt", delim_whitespace=True)
gg = gg[['ldmodel1','ldmodel5','ldmodel6']]
gg = gg.values.reshape(-1,3)

ld1ratio = [round(y_pred_test[i] /(gg[i,0]*1000),4) for i in np.arange(len(y_pred_test))]
ld5ratio = [round(y_pred_test[i] /(gg[i,1]*1000),4) for i in np.arange(len(y_pred_test))]
ld6ratio = [round(y_pred_test[i] /(gg[i,2]*1000),4) for i in np.arange(len(y_pred_test))]

In [7]:
with open("data_gamma_test.txt","w") as f:
    f.write('# Basic Learning Model - Calculating unknown test case of Average Radiative Width - Score:{0} \n'.format(round(test_score,4)))
    f.write('#Z   A  mlGg   mldG   ld1ratio   ld5ratio   ld6ratio \n')
    for i in range(0, len(y_pred_test)):
        f.write("{0}   {1}   {2}   {3}   {4}   {5}   {6}\n".format(arr_test_data[i,0],arr_test_data_inc[i],round(y_pred_test[i],4),round(np.sqrt(rf_err_test[i]),4),ld1ratio[i],ld5ratio[i],ld6ratio[i]))

In [8]:
a_inc = signal[['A']] + 1
arr_a_inc = a_inc.values.reshape(-1,1)
arr_a_inc = arr_a_inc.ravel()

In [9]:
with open("data_inc_gg.txt","w") as f:
#     f.write('# Basic Learning Model - Calculating unknown test case - Score:{0} \n'.format(round(test_score,4)))
#     f.write('#Z   A   mlD0   mlDd   ldratio1   ldratio5   ldratio6\n')
    for i in range(0, len(arr_a_inc)):
        f.write("{0}      {1}      {2} \n".format(arr_a_inc[i],arr_y_pred[i],arr_y_err[i]))